# System Init

In [1]:
from fov import FOV
from queue import Queue
import numpy as np
from matplotlib import pyplot as plt
#from napari import Viewer
import numpy as np
from queue import Queue
from useq import MDAEvent
from useq._channel import Channel
import pandas as pd
import random
from utils import ImgType, MetadataDict
#from stimulation import StimExtraParameters
from hardware import load_config
import os
import napari

from napari_micromanager import MainWindow

import skimage
import useq
from dmd import DMD

# from tracking import TrackerTrackpy
# from controller import Controller, Analyzer

from utils import create_folders
import numpy as np
import pymmcore_plus

import time

from useq._mda_event import SLMImage
import useq
import requests

mmc = pymmcore_plus.CMMCorePlus()
mmc.loadSystemConfiguration("E:\\MicroManagerConfigs\\Ti2CicercoConfig_w_DMD_21_w_ttl.cfg")

def wakeup_laser(lumencore_ip="192.168.201.200"):
    url = f"http://{lumencore_ip}/service/?command=WAKEUP"
    requests.get(url)
wakeup_laser()

In [52]:
wakeup_laser()

### DMD Full On

In [2]:
# %%capture
slm_dev = mmc.getSLMDevice()
slm_width = mmc.getSLMWidth(slm_dev)
slm_height = mmc.getSLMHeight(slm_dev)

event_slm_on = MDAEvent(slm_image=SLMImage(data=True))
mmc.mda.run([event_slm_on])# to only have fov of DMD 
mmc.setROI(150, 150, 1900, 1900)

DMD_CHANNEL_GROUP = "WF_DMD"
DMD_CALIBRATION_PROFILE = {"channel_group": "WF_DMD", "channel_config": "CyanStim", "device_name": "LedDMD", "property_name": "Cyan_Level", "power": 100}
mmc.setChannelGroup(channelGroup=DMD_CHANNEL_GROUP)

[01/15/25 20:52:06] INFO     MDA Started: GeneratorMDASequence()                                     ]8;id=928639;file://c:\Users\Niesen\.conda\envs\pymmc_w_stardist\lib\site-packages\pymmcore_plus\mda\_runner.py\_runner.py]8;;\:]8;id=655251;file://c:\Users\Niesen\.conda\envs\pymmc_w_stardist\lib\site-packages\pymmcore_plus\mda\_runner.py#329\329]8;;\

                    INFO     slm_image=SLMImage()                                                    ]8;id=258846;file://c:\Users\Niesen\.conda\envs\pymmc_w_stardist\lib\site-packages\pymmcore_plus\mda\_runner.py\_runner.py]8;;\:]8;id=705999;file://c:\Users\Niesen\.conda\envs\pymmc_w_stardist\lib\site-packages\pymmcore_plus\mda\_runner.py#290\290]8;;\

                    INFO     MDA Finished: GeneratorMDASequence()                                    ]8;id=774390;file://c:\Users\Niesen\.conda\envs\pymmc_w_stardist\lib\site-packages\pymmcore_plus\mda\_runner.py\_runner.py]8;;\:]8;id=152038;file://c:\Users\Niesen\.conda\envs\pymmc_w_stardist\lib\site-packages\pymmcore_plus\mda\_runner.py#416\416]8;;\

### Napari Micromanger User Interface

In [3]:
viewer = napari.Viewer()
mm_wdg = MainWindow(viewer)
viewer.window.add_dock_widget(mm_wdg)

In [ ]:
from pymmcore_widgets.mda import MDAWidget
mdawidget = MDAWidget(mmcore = mmc)
viewer.window.add_dock_widget(mdawidget)

In [31]:
mm_wdg._core_link.cleanup()

In [19]:
from napari_micromanager._core_link import CoreViewerLink
mm_wdg._core_link = CoreViewerLink(viewer, mmc)

## create a DF with all planned acquisitions and stimulations

In [17]:
df_acquire = pd.DataFrame(columns=['fov', 'timestep', 'time','time_experiment', 'treatment', 'acquired','stim', 'channels', 'channel_stim'])

path = "C:\\test\\"
create_folders(path,['stim','raw','labels','stim_mask','tracks','labels_rings','particles'])

time_between_frames = 10 #time in seconds between frames

timesteps = range(3)  # 0-20
channels = ['Red', 'Green']
channels_exposure = [200, 200]
channels_power = [48, 33]

cell_lines = ["optoFGFR1", "optoERK1", "optoERK1"]

# stim_exposures = [50, 100, 250]
stim_exposures = [100]
stim_timesteps = [[0]]  # list of timesteps
stim_profiles = [{"device_name": "LedDMD", "property_name": "Cyan_Level", "power": 10, "channel": "CyanStim"}]
stim_treatment = [{"stim_property": "global", "stim_profile": stim_profile, "stim_exposure": stim_exposure, "stim_timestep": stim_timestep} 
                  for stim_profile in stim_profiles for stim_exposure in stim_exposures for stim_timestep in stim_timesteps]
random.shuffle(stim_treatment)

for channel, power in zip(channels, channels_power):
    mmc.setProperty("Laser", f"{channel.upper()}_Intensity", power)
data_mda_fovs = None

Directory C:\test\stim already exists
Directory C:\test\raw already exists
Directory C:\test\labels already exists
Directory C:\test\stim_mask already exists
Directory C:\test\tracks already exists
Directory C:\test\labels_rings already exists
Directory C:\test\particles already exists


### Load points from file

In [19]:
import json
file = os.path.join(path, "test.json")
with open(file, "r") as f:
    data_mda_fovs = json.load(f)


### Using autogenerated points

In [6]:
fovs:list[FOV] = []
if data_mda_fovs is None:
    data_mda_fovs = mdawidget.value()
n_fovs = len(data_mda_fovs.stage_positions)
pts_per_well = data_mda_fovs.stage_positions.well_points_plan.num_points
n_wells = n_fovs // pts_per_well

for i, row in enumerate(data_mda_fovs.stage_positions):
    well_column = int(row.name.split('_')[0][1:])
    well_row = row.name.split('_')[0][0]
    well_id = i%pts_per_well
    cell_line = cell_lines[well_column-1]
    fov = FOV(pos=(row.x, row.y),
              index=i,
              name=row.name,
              path=path,
              metadata={"well_column": well_column, "well_row": well_row, "well_id": well_id, "cell_line": cell_line},
              treatment=stim_treatment[i%len(stim_treatment)],
              )
    fovs.append(fov)

AttributeError: 'list' object has no attribute 'stage_positions'

### Manually defined points

In [42]:
fovs:list[FOV] = []
if data_mda_fovs is None:
    data_mda_fovs = mdawidget.value().stage_positions
n_fovs = len(data_mda_fovs)
n_fovs_per_stim_condition = 2
stim_treatment_tot = stim_treatment * n_fovs_per_stim_condition
random.shuffle(stim_treatment_tot)

for i, row in enumerate(data_mda_fovs):
    fov = FOV(pos=(row["x"], row["y"]),
              index=i,
              name=str(i),
              path=path,
              metadata={},
              treatment=stim_treatment_tot[i],
              )
    fovs.append(fov)

### Generate timetable

In [43]:
time_per_fov = 5
n_fovs_simultaneously = time_between_frames // time_per_fov
start_time = 0

dfs = []
for fov in fovs:
    fov_group = fov.index // n_fovs_simultaneously
    start_time = fov_group * time_between_frames * len(timesteps)

    for timestep in timesteps:
        new_row = { 'fov_object': fov,
                    'fov':fov.index,
                    'name':fov.name,
                    'timestep': timestep,
                    'time': start_time + timestep*time_between_frames,
                    'treatment': fov.treatment,
                    'metadata': fov.metadata,
                    'acquired': False,
                    'stim': timestep in fov.treatment['stim_timestep'],
                    'channels': channels,
                    'channels_exposure':channels_exposure,
                    'channel_power': channels_power,
                    'stim_profile' : fov.treatment['stim_profile'],
                    'stim_exposure' : fov.treatment['stim_exposure'],
                    'fname' : f'{str(fov.index).zfill(3)}_{str(timestep).zfill(5)}',
                    }
        dfs.append(new_row)

df_acquire = pd.DataFrame(dfs)
df_acquire = df_acquire.sort_values(by=['time', 'fov'])
print(f"Total Experiment Time: {df_acquire['time'].max()}s")
df_acquire


Total Experiment Time: 20s


,fov_object,fov,name,timestep,time,treatment,metadata,acquired,stim,channels,channels_exposure,channel_power,stim_profile,stim_exposure,fname
0,<fov.FOV object at 0x000001FC6D6A64D0>,0,0,0,0,"{'stim_property': 'global', 'stim_profile': {'...",{},False,True,"[Red, Green]","[200, 200]","[48, 33]","{'device_name': 'LedDMD', 'property_name': 'Cy...",100,000_00000
1,<fov.FOV object at 0x000001FC6D6A64D0>,0,0,1,10,"{'stim_property': 'global', 'stim_profile': {'...",{},False,False,"[Red, Green]","[200, 200]","[48, 33]","{'device_name': 'LedDMD', 'property_name': 'Cy...",100,000_00001
2,<fov.FOV object at 0x000001FC6D6A64D0>,0,0,2,20,"{'stim_property': 'global', 'stim_profile': {'...",{},False,False,"[Red, Green]","[200, 200]","[48, 33]","{'device_name': 'LedDMD', 'property_name': 'Cy...",100,000_00002


## Run on system

In [20]:
mm_wdg._core_link.cleanup()

NameError: name 'mm_wdg' is not defined

In [ ]:
dmd = DMD(mmc, DMD_CALIBRATION_PROFILE)
dmd.calibrate(verbous=True)

In [11]:
controller.stop_run()

In [16]:
mmc.mda.cancel()

In [24]:
mmc.mda.events.frameReady.disconnect()

In [44]:
%load_ext autoreload
%autoreload 2

from add_frame import ImageProcessingPipeline
from segmentation import DummySegmentator
from stimulation import StimWholeFOV    
from controller import Controller, Analyzer
from tracking_trackpy import TrackerTrackpy
from dmd import DMD

segmentator = DummySegmentator()
stimulator = StimWholeFOV()
tracker = TrackerTrackpy()

pipeline = ImageProcessingPipeline(segmentator,stimulator,tracker)
analyzer = Analyzer(pipeline)
queue = Queue()
STOP = object()
import threading
def run_mda_in_a_thread(mmc_i, queue, stop_element):
    queue_sequence = iter(queue.get, stop_element)
    th = threading.Thread(target= mmc_i.mda.run, args=(queue_sequence, ))
    th.start()
    return th
run_mda_in_a_thread(mmc, queue, STOP)

controller = Controller(analyzer, mmc, queue, STOP)
controller.run(df_acquire)




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
fov_object                      <fov.FOV object at 0x000001FC6D6A64D0>
fov                                                                  0
name                                                                 0
timestep                                                             0
time                                                                 0
treatment            {'stim_property': 'global', 'stim_profile': {'...
metadata                                                            {}
acquired                                                         False
stim                                                              True
channels                                                  [Red, Green]
channels_exposure                                           [200, 200]
channel_power                                                 [48, 33]
stim_profile         {'device_name': 'LedDMD', 'property_nam

[01/15/25 21:35:41] INFO     MDA Started: GeneratorMDASequence()                                     ]8;id=11255;file://c:\Users\Niesen\.conda\envs\pymmc_w_stardist\lib\site-packages\pymmcore_plus\mda\_runner.py\_runner.py]8;;\:]8;id=866993;file://c:\Users\Niesen\.conda\envs\pymmc_w_stardist\lib\site-packages\pymmcore_plus\mda\_runner.py#329\329]8;;\

                    INFO     index={'t': 0, 'c': 0} channel=Channel(config='Red', group='WF_DMD')    ]8;id=176322;file://c:\Users\Niesen\.conda\envs\pymmc_w_stardist\lib\site-packages\pymmcore_plus\mda\_runner.py\_runner.py]8;;\:]8;id=124764;file://c:\Users\Niesen\.conda\envs\pymmc_w_stardist\lib\site-packages\pymmcore_plus\mda\_runner.py#290\290]8;;\
                             exposure=200.0 min_start_time=0.0 x_pos=-2058.2 y_pos=-8285.4                         
                             metadata={'fov_object': <fov.FOV object at 0x000001FC6D6A64D0>, 'fov':                
                             0, 'name': '0', 'timestep': 0, 'time': 0, 'treatment':                                
                             {'stim_property': 'global', 'stim_profile': {'device_name': 'LedDMD',                 
                             'property_name': 'Cyan_Level', 'power': 10, 'channel': 'CyanStim'},                   
                             'stim_exposure': 100, 'stim_timestep': [0]}, 'metadata': {},                          
                             'acquired': False, 'stim': True, 'channels': ['Red', 'Green'],                        
                             'channels_exposure': [200, 200], 'channel_power': [48, 33],                           
                             'stim_profile': {'device_name': 'LedDMD', 'property_name':                            
                             'Cyan_Level', 'power': 10, 'channel': 'CyanStim'}, 'stim_exposure':                   
                             100, 'fname': '000_00000', 'img_type': <ImgType.IMG_RAW: 1>,                          
                             'last_channel': False, 'channel': 'Red'}                                              

[01/15/25 21:36:01] INFO     index={'t': 0, 'c': 1} channel=Channel(config='Green', group='WF_DMD')  ]8;id=29682;file://c:\Users\Niesen\.conda\envs\pymmc_w_stardist\lib\site-packages\pymmcore_plus\mda\_runner.py\_runner.py]8;;\:]8;id=142701;file://c:\Users\Niesen\.conda\envs\pymmc_w_stardist\lib\site-packages\pymmcore_plus\mda\_runner.py#290\290]8;;\
                             exposure=200.0 min_start_time=0.0 x_pos=-2058.2 y_pos=-8285.4                         
                             metadata={'fov_object': <fov.FOV object at 0x000001FC6D6A64D0>, 'fov':                
                             0, 'name': '0', 'timestep': 0, 'time': 0, 'treatment':                                
                             {'stim_property': 'global', 'stim_profile': {'device_name': 'LedDMD',                 
                             'property_name': 'Cyan_Level', 'power': 10, 'channel': 'CyanStim'},                   
                             'stim_exposure': 100, 'stim_timestep': [0]}, 'metadata': {},                          
                             'acquired': False, 'stim': True, 'channels': ['Red', 'Green'],                        
                             'channels_exposure': [200, 200], 'channel_power': [48, 33],                           
                             'stim_profile': {'device_name': 'LedDMD', 'property_name':                            
                             'Cyan_Level', 'power': 10, 'channel': 'CyanStim'}, 'stim_exposure':                   
                             100, 'fname': '000_00000', 'img_type': <ImgType.IMG_RAW: 1>,                          
                             'last_channel': True, 'channel': 'Green'}                                             

Empty: 

In [24]:
calibration_profile["channel_group"]

'WF_DMD'

In [ ]:
mmc.unloadAllDevices()